# Initialization

In [ ]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd

from shapely.geometry import Point

In [ ]:
import sys
import subprocess

def installPackage(package):
    p = subprocess.run([sys.executable, "-m", "pip", "install", "-U", package], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print(p.stdout.decode())

requirements = ["folium", "leafmap", "ipyleaflet"]
for requirement in requirements:
    installPackage(requirement)

In [ ]:
# Bring in the additional libraries
import folium
import leafmap.foliumap as leafmap

# Function

In [ ]:
def overpass_output():
    overpass_url = "http://overpass-api.de/api/interpreter"
    # Bounding box coordinates for the Republic of Ireland
    bbox = "51.2,-10.5,55.3,-5.3"
    overpass_query = f"""
    [out:json][timeout:50];
    (
     node["amenity"="charging_station"]({bbox});
     way["amenity"="charging_station"]({bbox});
     relation["amenity"="charging_station"]({bbox});
    );
    out center;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    response.raise_for_status()
    return response.json()

In [ ]:
def extract_relevant_data(overpass_data):
    elements = overpass_data['elements']
    
    processed_data = []
    for element in elements:
        tags = element.get('tags', {})
        
        type_ = element.get('type', None)
        id_ = element.get('id', None)
        lat = element.get('lat', element.get('center', {}).get('lat', None))
        lon = element.get('lon', element.get('center', {}).get('lon', None))
        name = tags.get('name', None)
        addr_street = tags.get('addr:street', None)
        amenity = tags.get('amenity', None)
        capacity = tags.get('capacity', None)
        fee = tags.get('fee', None)
        access = tags.get('access', None)
        operator = tags.get('operator', None)
        ref = tags.get('ref', None)

        processed_data.append({
            'type': type_,
            'id': id_,
            'latitude': lat,
            'longitude': lon,
            'name': name,
            'address': addr_street,
            'amenity': amenity,
            'capacity': capacity,
            'fee': fee,
            'operator': operator,
            'ref': ref,
            'access': access
        })
    
    # Convert the list to a DataFrame
    df = pd.DataFrame(processed_data)
    return df

# Overpass Output

In [ ]:
# Converting json file to dataframe - implement this piece
raw_data = overpass_output()
charging_station_data = extract_relevant_data(raw_data)

In [ ]:
charging_station_data.sample(5)

In [ ]:
name_list = dict(charging_station_data.groupby('name', dropna=False)['id'].count().sort_values(ascending=False)[:13])
name_list

In [ ]:
name_list.keys()

In [ ]:
charge_data = (
    charging_station_data
    .assign(name_red=lambda df_:np.where(df_.name.isin(name_list.keys()),df_.name, 'Other')
    )
)
charge_data.head()

In [ ]:
charge_data.groupby(['name', 'name_red'], dropna=False)['id'].count().sort_values(ascending=False)[:20]

In [ ]:
charge_data.groupby('name_red', dropna=False)['id'].count().sort_values(ascending=False)

In [ ]:
# Combine two lists for icon
facility_cat = [i for i in charge_data['name_red'].unique()]
facility_cat

In [ ]:
# Hex colour list
hex_colours = ['#04F9E8', '#C44585', '#5722A1', '#D8F296', '#022EAB', '#E900C5', '#A7B3E3', '#64AD52', '#0B5348', '#5E7FDB', '#5CC8FF', '#D9E136', '#2C7786', '#D6DE0D']
purpose_colour1 = dict(zip(facility_cat, hex_colours[:len(facility_cat)]))

In [ ]:
purpose_colour1

In [ ]:
# Build the map
m1 = folium.Map(location=[charge_data.latitude.mean(), charge_data.longitude.mean()]
               ,zoom_start=3
               ,control_scale=True)

# Loop through each row in the dataframe
for i,row in charge_data.iterrows():
    # Setup the content of group
    iframe = folium.IFrame(f'Name: {str(row["name_red"])} \n Operator: {str(row["operator"])} \n address: {str(row["address"])}'
                          ,width=300
                          ,height=200)
    # Initialise the popup using the iframe
    popup = folium.Popup(iframe, max_width=650)

    # Icon colour
    try:
        icon_colour = purpose_colour1[row['name_red']]
    except:
        icon_colour = 'gray'
    
    # Add each row to the map
    folium.CircleMarker(location=[row['latitude'], row['longitude']]
                        ,radius=2
                        ,popup=popup
                        ,fill_color=icon_colour
                        ,color=icon_colour
                 ).add_to(m1)

# Display map
m1

## Setting Boundaries

In [ ]:
charging_station_data['geometry'] = charging_station_data.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)
charging_station_data = gpd.GeoDataFrame(charging_station_data, geometry='geometry', crs='EPSG:4326')

county_boundaries = gpd.read_file('.../Local_Authority.shapezip/Local_Authority.shp')

charging_station_data = gpd.sjoin(charging_station_data, county_boundaries, how="left")

In [ ]:
# Grouping up areas into county
dublin_county = ['South Dublin', 'Fingal', 'Dublin City', 'Dun Laoghaire-Rathdown']
waterford_county = ['Waterford County', 'Waterford City']
galway_county = ['Galway County', 'Galway City']
cork_county = ['Cork County', 'Cork City']
limerick_county = ['Limerick County', 'Limerick City']
tipperary_county = ['North Tipperary', 'South Tipperary']

def map_to_county(geographic):
    if geographic in dublin_county:
        return 'Dublin County'
    elif geographic in waterford_county:
        return 'Waterford County'
    elif geographic in galway_county:
        return 'Galway County'
    elif geographic in cork_county:
        return 'Cork County'
    elif geographic in limerick_county:
        return 'Limerick County'
    elif geographic in tipperary_county:
        return 'Tipperary County'
    else:
        return geographic

# Apply the function to the 'geographic' column
charging_station_data['County'] = charging_station_data['geographic'].apply(map_to_county)

# Select the columns for the output
output_data = charging_station_data[['type', 'id', 'latitude', 'longitude', 'nuts2Name', 'nuts3Name', 'County', 'geographic', 'address', 'name', 'capacity', 'access', 'operator']]

In [ ]:
output_data

In [ ]:
output_data.to_excel('.../Output/ROI Charging Stations.xlsx', index = False)

## Group By

In [ ]:
output_data['capacity'] = pd.to_numeric(output_data['capacity'], errors='coerce')

df_summ = (
    output_data
    .groupby(['County', 'access'], dropna=False)
    .agg({
        'capacity': ['sum', 'min', 'max', 'mean','count']
    })
)
df_summ

In [ ]:
df_summ.to_excel('.../Output/ROI Charging Station - Categorize.xlsx')

# OLD VERSION

Code below used to assess the json output file. Retained for peer review.

In [ ]:
# Create DataFrame for review
def create_df(input_dict=data.get('elements')):
    df = pd.DataFrame.from_dict(input_dict)
    # review tags list
    list_ch = df.tags.values.tolist()
    df2 = pd.DataFrame(list_ch)
    df3 = pd.concat([df, df2], axis=1)
    return (
        df3
        .assign(capacity=lambda df_:df_.capacity.astype(float))
        .drop(columns=['tags'])
    )

In [ ]:
# Convert the feature data types to allow for correct calculations
df = create_df()
df.head()

In [ ]:
# Review the outputs
df_summ = (
    df
    .groupby(['name', 'access'], dropna=False)
    .agg({
        'capacity': ['sum', 'min', 'max', 'mean','count']
    })
)
df_summ

### Overpy details

In [ ]:
pip install overpy

In [ ]:
import overpy
import requests
import json
import geojson
import pandas as pd
import numpy as np

In [ ]:
def overpass_output(dist=5000, lat=53.340483, lng=-6.257701):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json][timeout:25];
    (
     node["amenity"="charging_station"](around:{dist},{lat},{lng});
    );
    out geom;
    """
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    response.raise_for_status()
    return response.json()

In [ ]:
# Converting json file to dataframe - implement this piece
data_check = overpass_output()['elements']
data_check
df_ch = pd.json_normalize(data_check)
df_ch.head()

In [ ]:
# df_ch.columns
# df_ch.dtypes

In [ ]:
# Review the outputs
df_summ = (
    df_ch
    .assign(capacity=lambda df_:df_['tags.capacity'].astype(float))
    .groupby(['tags.name', 'tags.access'], dropna=False)
    .agg({
        'capacity': ['sum', 'min', 'max', 'mean','count']
    })
)
df_summ

In [ ]:
data = overpass_output()
data